In [1]:
import pandas as pd
import re

In [82]:

df_sandp500_ECT_merge_without_spilt=pd.read_csv("D:/Script/sandp500_conference_earning_transcript.csv")

'''Spilt Presentation, Q&A, Company Participants, Conference Call Participants'''

def find_marker(text, markers):
    """在文本中找到第一个出现的标记，并返回该标记和它的位置"""
    first_index = len(text)  # 初始化一个极大的索引值作为比较基础
    first_marker = None
    for marker in markers:
        index = text.find(marker)
        if index != -1 and (first_marker is None or index < first_index):
            first_index = index
            first_marker = marker
    return first_marker, first_index

def extract_participants(text, start_markers, end_markers):
    """从可能的多个起始和结束标记中提取参与者列表"""
    start_marker, start_index = find_marker(text, start_markers)
    if start_marker is None:
        return ['No sample']  # 如果没有找到起始标记，直接返回空列表

    start_index += len(start_marker)  # 调整开始位置到标记之后

    end_marker, end_index = find_marker(text[start_index:], end_markers)
    if end_marker is not None:
        end_index += start_index  # 调整结束索引为相对于整个文本的位置
    else:
        end_index = None

    participant_section = text[start_index:end_index if end_index is not None else None]

    # 使用正则表达式查找名字，适应名字中可能包含的点和连字符
    '''20240418這個可以處理超多'''
    pattern = re.compile(r'(\b[\w’\'¡¦.]+(?:\s[A-Z]\w*\.)?\s[A-Z][’\'\w]+)(?=\s(?:[–-]|¡V))')

    participants = [match.group(1).strip() for match in pattern.finditer(participant_section)]
    
    return participants
def split_presentation_QandA(text):
    # 检查文本是否为空
    if pd.notna(text):
        # 提取 Company Participants 和 Conference Call Participants
        clean_text = text.replace('\n', ' ')
        company_participants = extract_participants(clean_text, ["Company Participants", "Executives","Corporate Participants","Company Representatives"], ["Conference Call Participants", "Analysts"])
        call_participants = extract_participants(clean_text, ["Conference Call Participants", "Analysts"], ["Operator"])

        # 初始化段落
        presentation = []
        QandA = []
        in_presentation = True  # Initially, everything is considered part of the presentation
        in_QandA = False  # This will turn true once we hit 'Question-and-Answer Session'

        # 分析文本，提取演示和问答部分
        lines = text.split('\n')
        for line in lines:
            if 'Question-and-Answer Session' in line:
                # 找到 'Question-and-Answer Session', 标记问答开始
                in_presentation = False
                in_QandA = True
                continue  # 跳过这一行
            
            if in_presentation:
                presentation.append(line)
            elif in_QandA:
                QandA.append(line)
        
        # 将提取的信息添加到结果中
        return (' '.join(presentation), ' '.join(QandA), company_participants, call_participants)
    else:
        # 如果文本为空，则返回空字符串和空列表
        return "", "", [], []

# 应用函数到 DataFrame 的每行，并创建新列
df_sandp500_ECT_merge_without_spilt[['Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants']] = \
    df_sandp500_ECT_merge_without_spilt['text'].apply(
        lambda txt: pd.Series(split_presentation_QandA(txt))
    )



In [47]:
# Check Nan/Missing value
nan_pre_index = df_sandp500_ECT_merge_without_spilt[df_sandp500_ECT_merge_testagain['CompanyParticipants'].isna()].index
nan_pre_index_list = nan_pre_index.tolist()
print(nan_pre_index_list)

[]


In [83]:
df_sandp500_ECT_merge_without_spilt.to_csv("D:/Script/sandp500_conference_earning_transcript_participants.csv", index=False)

In [85]:
# Since file is too huge, filter with only Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants', 'title' part
df_filtered = df_sandp500_ECT_merge_without_spilt[['Presentation', 'QandA', 'CompanyParticipants', 'CallParticipants', 'title']]
df_filtered.to_csv("D:/Script/sandp500_pre_qanda_participantstranscript.csv", index=False)